In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    api_key = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version = "2023-03-15-preview",
    azure_endpoint = os.getenv("AZURE_BASE_URL"),
    temperature = 0.2, 
    model = "gpt-4o-mini"
)

In [8]:
from langchain.tools import tool

# @tool
# def multiplynumbers(input: str) -> int:
#     ''' 
#     multiplies two number a and b. input should be space-separated integers.
#     '''
#     #parse the input to extract number
#     a = int(input.split()[0])
#     b = int(input.split()[1])
#     return a*b


# You can also customize the tool name and JSON args by passing them into the tool decorator.
from langchain.pydantic_v1 import BaseModel, Field

class MultiplyInput(BaseModel):
    input: str = Field(description="search query")


@tool("multiply-tool", args_schema=MultiplyInput, return_direct=True)
def multiplynumbers(input: str) -> str:
    ''' 
    multiplies two number a and b. input should be space-separated integers.
    '''

    #parse the input to extract number
    a = int(input.split()[0])
    b = int(input.split()[1])
    return a*b


In [12]:
from langchain.agents import initialize, AgentType

agent = initialize.initialize_agent(
    tools = [multiplynumbers],
    llm = llm,
    agent_type = AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True
)

In [14]:
# Query the agent
query = "What is the result of multiplying 6 and 7?"
response = agent.invoke(query)
print(response)



> Entering new AgentExecutor chain...
To find the result of multiplying 6 and 7, I need to use the multiply-tool to perform the multiplication. 

Action: multiply-tool  
Action Input: "6 7"  
Observation: 42


> Finished chain.
{'input': 'What is the result of multiplying 6 and 7?', 'output': 42}


In [16]:
@tool
def addnumbers(input:str) -> int:
    ''' 
    Adds two number a and b.  input should be space-separated integers.
    '''

    #parse the input to extract number
    a = int(input.split()[0])
    b = int(input.split()[1])
    return a+b 


In [18]:
agent = initialize.initialize_agent(
    tools = [multiplynumbers, addnumbers],
    llm = llm,
    agent_type = AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    verbose = True
)

In [19]:
query_multiply = "What is the result of multiplying 6 and 7?"
agent.invoke(query_multiply)



> Entering new AgentExecutor chain...
To find the result of multiplying 6 and 7, I will use the multiply-tool function. 

Action: multiply-tool  
Action Input: "6 7"  
Observation: 42


> Finished chain.


{'input': 'What is the result of multiplying 6 and 7?', 'output': 42}

In [22]:
response = agent.invoke(query_multiply)
response



> Entering new AgentExecutor chain...
To find the result of multiplying 6 and 7, I need to use the multiply-tool function. 

Action: multiply-tool  
Action Input: "6 7"  
Observation: 42


> Finished chain.


{'input': 'What is the result of multiplying 6 and 7?', 'output': 42}

In [23]:
query_addition = "add 6 and 7?"
agent.invoke(query_addition)



> Entering new AgentExecutor chain...
To find the sum of 6 and 7, I need to use the addition tool.  
Action: addnumbers  
Action Input: "6 7"  
Observation: 13
Thought:I now know the final answer.  
Final Answer: 13

> Finished chain.


{'input': 'add 6 and 7?', 'output': '13'}

In [24]:
# Agent Example 2

@tool
def get_weather(location:str)->str:
    ''' 
    Provides the weather information for a given location. Input should be name of a location. Return the weather information directly without performing any further actions.
    '''

    return f'The weather in {location} is sunny with temperature of 25°C.'

In [25]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables = ["input", "agent_scratchpad"],
    template = ''' 
        You are an assistant that can answer questions directly or use tools when necessary.
        If a question requires specific information, like weather details, extract the necessary
        information (such as location) and use the appropriate tool. Otherwise, answer directly.
        {agent_scratchpad}
        Question: {input}
        Answer:
    '''
)

In [26]:
agent = initialize.initialize_agent(
    tools = [get_weather],
    llm = llm,
    agent_type = AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    verbose = True,
    prompt_template = prompt,
    handling_parsing_errors = True
)

In [28]:
query_general = "help me with bsics of maths."
agent.invoke(query_general)



> Entering new AgentExecutor chain...
I need to provide information about the basics of math. This doesn't require a weather check, so I can summarize key concepts instead. 

Final Answer: The basics of math include addition, subtraction, multiplication, and division. Other fundamental concepts are fractions, decimals, percentages, and basic geometry. Understanding these concepts forms the foundation for more advanced topics in mathematics.

> Finished chain.


{'input': 'help me with bsics of maths.',
 'output': 'The basics of math include addition, subtraction, multiplication, and division. Other fundamental concepts are fractions, decimals, percentages, and basic geometry. Understanding these concepts forms the foundation for more advanced topics in mathematics.'}

In [29]:
query_general = "how is the weather in new delhi."
agent.invoke(query_general)



> Entering new AgentExecutor chain...
I need to find the current weather information for New Delhi.  
Action: get_weather  
Action Input: "New Delhi"  
Observation: The weather in New Delhi is sunny with temperature of 25°C.
Thought:I now know the final answer  
Final Answer: The weather in New Delhi is sunny with a temperature of 25°C.

> Finished chain.


{'input': 'how is the weather in new delhi.',
 'output': 'The weather in New Delhi is sunny with a temperature of 25°C.'}

In [30]:
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        print("Goodbye!")
        break

    response_general = agent.run(user_input)
    print("Agent:", response_general)



> Entering new AgentExecutor chain...
I need to find out the current weather in Bengaluru.  
Action: get_weather  
Action Input: "Bengaluru"  
Observation: The weather in Bengaluru is sunny with temperature of 25°C.
Thought:I now know the final answer  
Final Answer: The weather in Bengaluru is sunny with a temperature of 25°C.

> Finished chain.
Agent: The weather in Bengaluru is sunny with a temperature of 25°C.
Goodbye!


In [64]:
# Agent Example 3
from langchain_core.output_parsers import StrOutputParser

decision_prompt = PromptTemplate(
    input_variable = ["query"],
    template=(
    ''' 
        If the query requires specific information about weather, return 'True'. Otherwise, return the answer based on what is asked in the question.
        Do not return anything else other than True if the query is asking about the weather
        Query: {query}
        Answer:
    '''
    )
)

# decision_chain = LLMChain(
#   prompt = decision_prompt  
#   llm=llm,
# )

decision_chain = decision_prompt | llm | StrOutputParser()

In [74]:
def should_use_tool(query:str)->str:
    decision = decision_chain.invoke({
        "query": query
        })
    return decision.lower().strip()

def custom_agent(query:str)->str:
    response_llm = should_use_tool(query)
    if response_llm == "true":
        agent = initialize.initialize_agent(
                    tools = [get_weather],
                    llm = llm,
                    agent_type = AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
                    verbose = True,
                    prompt_template = prompt,
                    handling_parsing_errors = True
                )
        response = agent.invoke(query)

    else:
        response = response_llm

    return response

In [75]:
query = "Who is the president of United States ?"
response_general = custom_agent(query)
print("Agent: ", response_general)

Agent:  the current president of the united states is joe biden.


In [76]:
# Example where the LLM should decide to use a tool
query_weather = "What is the weather in Delhi?"
response_weather = custom_agent(query_weather)
print("Agent:", response_weather)



> Entering new AgentExecutor chain...
I need to find out the current weather in Delhi.  
Action: get_weather  
Action Input: "Delhi"  
Observation: The weather in Delhi is sunny with temperature of 25°C.
Thought:I now know the final answer  
Final Answer: The weather in Delhi is sunny with a temperature of 25°C.

> Finished chain.
Agent: {'input': 'What is the weather in Delhi?', 'output': 'The weather in Delhi is sunny with a temperature of 25°C.'}


In [83]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key = "chat_history"
)

agent = initialize.initialize_agent(
    tools = [get_weather],
    llm = llm,
    agent_type = AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    verbose = True,
    prompt_template = prompt,
    handling_parsing_errors = True,
    memory = memory
)

In [84]:
# Example interactions with the agent
query1 = "What is the weather in Delhi?"
agent.invoke({
    "input": query1
})



> Entering new AgentExecutor chain...
I need to find out the current weather in Delhi.  
Action: get_weather  
Action Input: "Delhi"  
Observation: The weather in Delhi is sunny with temperature of 25°C.
Thought:I now know the final answer  
Final Answer: The weather in Delhi is sunny with a temperature of 25°C.

> Finished chain.


{'input': 'What is the weather in Delhi?',
 'chat_history': '',
 'output': 'The weather in Delhi is sunny with a temperature of 25°C.'}

In [85]:
query2 = "And what about New York?"
agent.invoke({
    "input":query2
    })



> Entering new AgentExecutor chain...
I need to find out the weather information for New York.  
Action: get_weather  
Action Input: "New York"  
Observation: The weather in New York is sunny with temperature of 25°C.
Thought:I now know the final answer  
Final Answer: The weather in New York is sunny with a temperature of 25°C.

> Finished chain.


{'input': 'And what about New York?',
 'chat_history': 'Human: What is the weather in Delhi?\nAI: The weather in Delhi is sunny with a temperature of 25°C.',
 'output': 'The weather in New York is sunny with a temperature of 25°C.'}

In [86]:
print(memory.buffer)

Human: What is the weather in Delhi?
AI: The weather in Delhi is sunny with a temperature of 25°C.
Human: And what about New York?
AI: The weather in New York is sunny with a temperature of 25°C.
